# Agenda

1. Data set that we're using -- NYC taxis
2. Load the data set and perform some basic operations with it
3. Use the filenames to set column information
4. Do some more analysis, comparing pandemic to non-pandemic
5. Be a bit smarter about reading + using data from the data set to set column information
6. Was this really smarter?

In [1]:
!ls *.csv

nyc_taxi_2019-01.csv  nyc_taxi_2020-01.csv
nyc_taxi_2019-07.csv  nyc_taxi_2020-07.csv


In [2]:
!ls -l *.csv

-rw-r--r-- 1 reuven staff 687088084 Sep 13 18:28 nyc_taxi_2019-01.csv
-rw-r--r-- 1 reuven staff 584387609 Sep 13 18:28 nyc_taxi_2019-07.csv
-rw-r--r-- 1 reuven staff 593610736 Sep 13 18:28 nyc_taxi_2020-01.csv
-rw-r--r-- 1 reuven staff  73326707 Sep 13 18:28 nyc_taxi_2020-07.csv


In [3]:
import pandas as pd
from pandas import Series, DataFrame

In [12]:
# to load CSV data into pandas, we can use pd.read_csv

import numpy as np

df = pd.read_csv('nyc_taxi_2019-01.csv',
                usecols=['passenger_count', 'trip_distance', 'total_amount',
                        'payment_type'],
                dtype={'passenger_count':np.int8,
                      'payment_type':np.int8,
                      'trip_distance'})

In [13]:
df.head() 

,passenger_count,trip_distance,payment_type,total_amount
0,1,1.5,1,9.95
1,1,2.6,1,16.30
2,3,0.0,1,5.80
3,5,0.0,2,7.55
4,5,0.0,2,55.55


In [14]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667792 entries, 0 to 7667791
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   passenger_count  int8   
 1   trip_distance    float64
 2   payment_type     int8   
 3   total_amount     float64
dtypes: float64(2), int8(2)
memory usage: 131.6 MB
